<a href="https://colab.research.google.com/github/karthikravi123/genai_samples/blob/main/genai_project_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
 !pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.8 MB/s eta 0:00:00


In [18]:
from groq import Groq

In [83]:
client = Groq(api_key = '')

In [20]:
##we create bot which will give smart answers based on data

In [21]:
path="/content/tips.csv"


In [22]:
import pandas as pd
import json

In [23]:
def load_csv(path):
  df = pd.read_csv(path)
  return df

In [61]:
# df =load_csv(path)

In [27]:
def chunk_dataframe(df,chunk_size=5):
  chunks = []
  for i in range(0,len(df),chunk_size):
    chunk = df.iloc[i:i+chunk_size]
    chunks.append(chunk.to_json())
  return chunks

In [62]:
# chunks = chunk_dataframe(df)

In [29]:
##create embeddings and saving vector databse

In [30]:
   !pip install faiss-cpu


In [31]:
   !pip install sentence-transformers

In [32]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [33]:
model = SentenceTransformer("all-MIniLM-L6-v2")

In [34]:
##embedding and saving it to vector database -- RAG (retrieve)
def create_faiss_index(chunks):
  embeddings = model.encode(chunks)
  dim = embeddings.shape[1]
  index = faiss.IndexFlatL2(dim)
  index.add(np.array(embeddings))
  return index,chunks,embeddings

In [63]:
# index,chunks,embeddings = create_faiss_index(chunks)

In [36]:
embeddings

array([[-0.00906115,  0.0952032 ,  0.03625135, ..., -0.01825793,
        -0.02354873,  0.01804829],
       [-0.01475511,  0.10058314,  0.04442241, ..., -0.02812134,
        -0.02520751,  0.0206052 ],
       [-0.01854177,  0.10324341,  0.03327687, ..., -0.0217    ,
        -0.02785634,  0.02000053],
       ...,
       [-0.02334326,  0.07904304,  0.00455605, ..., -0.0242606 ,
        -0.02082959,  0.02192292],
       [-0.01070084,  0.07555556, -0.00012428, ..., -0.01385481,
        -0.01458977,  0.02375435],
       [-0.00838208,  0.08112466,  0.02615254, ..., -0.02487729,
        -0.01395411,  0.00806544]], dtype=float32)

In [37]:
##performing retrieval

def retriev_relevant_chunks(query,index,chunks,embeddings,k=2):
  query_vec = model.encode([query])
  distance,indices = index.search(query_vec,k)
  return [chunks[i]  for i in indices[0]]


In [79]:
def generate_response_from_groq(context,query):
  prompt = f"""
  you are a smart data assistant.Here's some context from a csv file
  {context} \n\n
  Questions:
  {query}\n
  Answer
   """

  completion = client.chat.completions.create(
       model = 'llama-3.3-70b-versatile',
       messages = [
           {"role": 'system',
            "content": "your are a data analyst assistant"},
           {
               "role":"user",
               "content": 'prompt'
           },

       ],
      temperature = 0.2
   )

  return completion.choices[0].message.content

In [78]:
def generate_insights(df):
  #stats = df.describe(include='all').to_string()
  stats = df.describe(include='all').to_string()
  prompt = f"""
  Here is a dataset summary :\n{stats}\n\nGenerate 3 smart insights or trends that you observe
   """

  completion = client.chat.completions.create(
       model = 'llama-3.3-70b-versatile',
       messages = [
           {"role": 'system',
            "content": 'prompt'}
       ])

  return completion.choices[0].message.content

In [82]:
if __name__ == "__main__":


  path="/content/tips.csv"

  df =load_csv(path)
  chunks = chunk_dataframe(df)
  index,chunk,embeddings = create_faiss_index(chunks)

  while True:
    query = input("Ask a question (or type quit or exit):")
    if query.lower() == "exit" :
      break
    elif query.lower() == "quit":
      print("Generating smart insights")
      print(generate_insights(df))
    else:
      content = "\n\n".join(retriev_relevant_chunks(query,index,chunk,embeddings))
      answer = generate_response_from_groq(content,query)
      print("answer",answer)



Ask a question (or type quit or exit):what is average tips?
answer I'm ready to help with any data analysis tasks. What kind of assistance do you need? Do you have a specific dataset you'd like me to work with, or a particular question you'd like me to help answer? 

Here are some examples of tasks I can help with:

1. Data cleaning and preprocessing
2. Data visualization (e.g. creating plots, charts, etc.)
3. Statistical analysis (e.g. hypothesis testing, regression, etc.)
4. Data mining and machine learning
5. Data interpretation and insights

Please let me know how I can assist you.
Ask a question (or type quit or exit):1
answer I'm ready to help with any data-related tasks. What kind of assistance do you need? Do you have a specific dataset you'd like me to analyze, or perhaps a question about data visualization or statistical modeling? 

Here are some areas where I can assist:

1. **Data Analysis**: I can help with descriptive statistics, data cleaning, and data transformation.
2.

KeyboardInterrupt: Interrupted by user